In [6]:
import pandas as pd
from lsst.daf.butler import Butler

In [2]:
!pwd

/pscratch/sd/s/shl159/projects/pDIA/dia_improvement/devel


In [3]:
def remove_src_with_flags(src_df, flag_list=None):
    flags = src_df.loc[:, flag_list]
    keep_id = (flags.sum(axis=1) == 0).to_numpy()
    return src_df[keep_id].copy().reset_index(drop=True)

def keep_src_with_flags(src_df, flag_list=None):
    flags = src_df.loc[:, flag_list]
    keep_id = (flags.sum(axis=1) > 0).to_numpy()
    return src_df[keep_id].copy().reset_index(drop=True)

In [7]:
butler = Butler("/global/cfs/cdirs/lsst/production/gen3/DECam/DECAT/repo",collections="u/rearmstr/DECam/runs/2021A-0113/w_2023_34/diff_2023A-716082_template",)

In [8]:
"""
# Full exposure list
exp_list = list(butler.registry.queryDimensionRecords("exposure"))

# filter on the data I processed difference images for which are in the 2021A-0113 semester for this collection
filt_exp_list = [
    exp
    for exp in exp_list
    if ((exp.science_program == "2021A-0113") & (exp.observation_type == "science"))
]

print(len(exp_list), len(filt_exp_list))

exp_info = pd.DataFrame()
for i in range(len(filt_exp_list)):
    info = filt_exp_list[i].toDict()
    info = pd.DataFrame([info])
    exp_info = pd.concat([exp_info, info], ignore_index=True)
    
exp_info.to_csv('decat_exp_info.csv', index=False)
""";

In [9]:
exp_info = pd.read_csv('./decat_exp_info.csv')

min_det = 1
max_det = 62
det_list = [d for d in range(min_det, max_det+1)]

dipole_counts = pd.DataFrame()
for i in range(len(exp_info)):
    print(i)
    copied_df = exp_info.iloc[[i] * max_det].copy().reset_index(drop=True)
    copied_df['detector'] = det_list
    n_dipoles_list = []
    for j, row in copied_df.iterrows():
        exp_id = int(row['id'])
        det = int(row['detector'])
        diff_src_id_list = list(butler.registry.queryDatasets("goodSeeingDiff_diaSrc",where=f"exposure={exp_id} and detector={det} and instrument='DECam'"))
        if len(diff_src_id_list) == 0:
            n_dipoles_list.append(-1)
        else:
            diff_src_id = diff_src_id_list[0]
            diff_src = butler.get(diff_src_id).asAstropy().to_pandas()
            nosat = remove_src_with_flags(diff_src, flag_list=['base_PixelFlags_flag_saturated', 'base_PixelFlags_flag_saturatedCenter'])
            dipole = keep_src_with_flags(nosat, flag_list=['ip_diffim_DipoleFit_flag_classification', 'ip_diffim_DipoleFit_flag_classificationAttempted'])
            n_dipoles_list.append(len(dipole))
    copied_df['n_dipoles'] = n_dipoles_list
    dipole_counts = pd.concat([dipole_counts, copied_df], ignore_index=True)
    
dipole_counts.to_csv('./decat_dipole_counts.csv', index=False)        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27